In [1]:
import json
import xml.etree.ElementTree as et
import os
# import cv2
import numpy as np
import sys
import pandas as pd

In [ ]:
# %reload_ext autoreload
# %autoreload 2

## Dividing data to crossing and not crossing (2nd)

In [147]:
#### for filtering crossing and not crossing ###
def get_cnc_id(data):
    crossing_id_frame={}
    ncrossing_id=[]
    person_ids=[*data]
    for person_id in person_ids:
        frames=[*data[person_id]]
        count=0
        if (data[person_id][frames[0]]['c_nc']=='crossing'): # ignoring crossing from the start person.
            continue

        crossing=False
        nc_candidate=False

        for frame in frames:
            c_nc=data[person_id][frame]['c_nc']
            if c_nc=='not-crossing':
                nc_candidate=True
            elif c_nc=='crossing':
                crossing=True
                nc_candidate=False
                last_frame=int(frame)
                break
        if crossing==True:
            crossing_id_frame[person_id]=str(last_frame)
        elif nc_candidate==True:
            ncrossing_id.append(person_id)
    return crossing_id_frame,ncrossing_id
    

In [219]:
for i in range(5,347):
    video_num='0000'+str(i)
    video_num=video_num[-4:]
    json_path='./final_annot/data_with_keypoints/video_'+video_num+'.json' #'./annot_json/video_'+video_num+'.json' 

    with open(json_path,'r') as f:
        data=json.load(f)

    crossing_id_frame,ncrossing_id=get_cnc_id(data)
    
    if len(crossing_id_frame)!= 0:
        new_data_crossing={}
        for person_id in crossing_id_frame:
            frames=[*data[person_id]]
            frame_dict={}
            for frame in frames:
                box=data[person_id][frame]['bbox']
                c_nc=data[person_id][frame]['c_nc']
                keypoints=data[person_id][frame]['keypoints']
                frame_dict[frame]={'bbox':box,'c_nc':c_nc,'keypoints':keypoints}
                if frame==crossing_id_frame[person_id]:
                    break
            new_data_crossing[person_id]=frame_dict

        json_write_path_crossing='./final_annot/crossing_candidate/video_'+video_num+'.json'

        #for writing json
        with open(json_write_path_crossing,'w') as f:
            json.dump(new_data_crossing,f,indent=2)

    if len(ncrossing_id)!= 0:
        new_data_ncrossing={}
        for person_id in ncrossing_id:
            frames=[*data[person_id]]
            frame_dict={}
            for frame in frames:
                box=data[person_id][frame]['bbox']
                c_nc=data[person_id][frame]['c_nc']
                keypoints=data[person_id][frame]['keypoints']
                frame_dict[frame]={'bbox':box,'c_nc':c_nc,'keypoints':keypoints}
            new_data_ncrossing[person_id]=frame_dict

        json_write_path_ncrossing='./final_annot/ncrossing_candidate/video_'+video_num+'.json'

        ##for writing json
        with open(json_write_path_ncrossing,'w') as f:
            json.dump(new_data_ncrossing,f,indent=2)

In [214]:
crossing_id_frame,ncrossing_id=get_cnc_id(data)

In [215]:
ncrossing_id

['person_0']

In [216]:
crossing_id_frame

{}

In [116]:
[*data[person_id]][0]

'18'

## Getting keypoints and creating new json with keypoints for each frame(1st)

In [60]:
def iou_end(bbox, candidates):
    """Computes intersection over union.

    Parameters
    ----------
    bbox : ndarray
        A bounding box in format `(top left x, top left y, width, height)`.
    candidates : ndarray
        A matrix of candidate bounding boxes (one per row) in the same format
        as `bbox`.

    Returns
    -------
    ndarray
        The intersection over union in [0, 1] between the `bbox` and each
        candidate. A higher score means a larger fraction of the `bbox` is
        occluded by the candidate.

    """
    bbox_tl, bbox_br = bbox[:2], bbox[:2] + bbox[2:]
    candidates_tl = candidates[:, :2]
    candidates_br = candidates[:, :2] + candidates[:, 2:]

    tl = np.c_[np.maximum(bbox_tl[0], candidates_tl[:, 0])[:, np.newaxis],
               np.maximum(bbox_tl[1], candidates_tl[:, 1])[:, np.newaxis]]
    br = np.c_[np.minimum(bbox_br[0], candidates_br[:, 0])[:, np.newaxis],
               np.minimum(bbox_br[1], candidates_br[:, 1])[:, np.newaxis]]
    wh = np.maximum(0., br - tl)

    area_intersection = wh.prod(axis=1)
    area_bbox = bbox[2:].prod()
    area_candidates = candidates[:, 2:].prod(axis=1)
    return area_intersection / (area_bbox + area_candidates - area_intersection)

In [87]:
###### for keypoints openpose json#####
def calculate_boxes(keypoints):
    keypoints=keypoints[np.invert(keypoints[:,2]==0)]
    #keypoints=keypoints[np.all(keypoints != 0, axis=1)] # for comparing if all three values in a row is equal to zero. 
    x1=keypoints[:,0].min()
    y1=keypoints[:,1].min()
    x2=keypoints[:,0].max()
    y2=keypoints[:,1].max()
    width=x2-x1
    height=y2-y1
    height=height+(0.08*height)
    y1=max((y1-(0.08*height)),0)
    bbox=[x1,y1,width,height]
    return bbox

def get_keypoints_and_boxes(file_name):
    bboxes=[]
    keypoints_lists=[]
    with open (file_name) as f:
        data= json .load(f)
    if len(data['people'])==0:
        bboxes.append([0.0,0.0,0.0,0.0])
        keypoints_lists.append(np.zeros((25,2)).tolist())
    else:
        for i in range(len(data['people'])):
            keypoints=data['people'][i]['pose_keypoints_2d']
            keypoints=np.array(keypoints)
            keypoints=keypoints.reshape(25,3)
            keypoints_array=keypoints.copy()
            keypoints=keypoints.tolist()
            keypoints_lists.append(keypoints_array[:,:2].tolist())

            bbox=calculate_boxes(keypoints_array)
            bboxes.append(bbox)
    return bboxes,keypoints_lists

def convert_to_tlwh(bbox):
    x1=float(bbox[0])
    y1=float(bbox[1])
    x2=float(bbox[2])
    y2=float(bbox[3])
    width=x2-x1
    height=y2-y1
    box=[x1,y1,width,height]
    return box

def convert_to_tlbr(bbox):
    x1=float(bbox[0])
    y1=float(bbox[1])
    width=float(bbox[2])
    height=float(bbox[3])
    x2=x1+width
    y2=y1+height
    box=[x1,y1,x2,y2]
    return box

In [86]:
##### for openpose keypoints and jaad boxes both ######
# def get_head_cord(bbox,boxes,head_cords):
#     '''
#     Comparing tracked boxes with original boxes based on iou 
#     and getting head co-ordinate of that person in paricular bounding box.
#     Note: head_cord in'head_cords' for each person is in same position as bounding box in 'boxes'  
#     '''
#     iou_values=iou_end(bbox, boxes)
#     iou_dict={'position':np.argmax(iou_values),'iou_value':np.max(iou_values)}
#     if iou_dict['iou_value'] >0.80:
#         head_cord=head_cords[iou_dict['position']]
#     else:
#         head_cord=(None,None)
#     return head_cord

def get_keypoints(bbox,boxes,keypoints_lists):
    '''
    Comparing tracked boxes with original boxes based on iou 
    and getting head co-ordinate of that person in paricular bounding box.
    Note: head_cord in'head_cords' for each person is in same position as bounding box in 'boxes'  
    '''
    bbox=np.array(bbox)
    boxes=np.array(boxes)
    iou_values=iou_end(bbox, boxes)
    iou_dict={'position':np.argmax(iou_values),'iou_value':np.max(iou_values)}
    if iou_dict['iou_value'] >0.10: #0.90 # infact change this to 0 to take any highest probablity value
        keypoints_list=keypoints_lists[iou_dict['position']]
    else:
        keypoints_list=np.zeros((25,2)).tolist() # make 25*2 matrix of zeros or think...
    return keypoints_list

In [91]:
# for jaad json
for i in range(1,347):
    video_num='0000'+str(i)
    video_num=video_num[-4:]
    json_path='./annot_json/video_'+video_num+'.json'

    with open(json_path,'r') as f:
        jaad_data=json.load(f)

    #for keypoints json
    total_frame =len(os.listdir('./nbn_output/video_'+video_num))

    for frame_num in range(total_frame):
        if len(jaad_data)==0:
            break
        file_num='000000000000'+str(frame_num)
        file_num=file_num[-12:]
        file_name='./nbn_output/video_'+video_num+'/video_'+video_num+'_'+file_num+'_keypoints.json'
        with open(json_path,'r') as f:
            keypoints_data=json.load(f)

        #getting bbox from jaad data dictionary for every person in this frame_num
        jaad_bboxes={}
        for person_id in range (len(jaad_data)):
            keys=[*jaad_data]
            try:
                jaad_bbox_tlbr=jaad_data[keys[person_id]][str(frame_num)]['bbox']
                jaad_bbox=convert_to_tlwh(jaad_bbox_tlbr)
            except:
                continue
            jaad_bboxes[keys[person_id]]=jaad_bbox

        pose_bboxes,keypoints_lists=get_keypoints_and_boxes(file_name) # here pose_bboxes is list of list and one bbox is in the form x,y,w,h
        for person_id in jaad_bboxes:
            try:
                keypoints_list=get_keypoints(jaad_bboxes[person_id],pose_bboxes,keypoints_lists) #here 1 keypoints_list can be related to more than 1 id incase of congestion
            except:
                print(file_name)
                print(jaad_bboxes[person_id])
                print('....')
                print(pose_bboxes)
                sys.exit('got exception')
            jaad_data[person_id][str(frame_num)]['keypoints']=keypoints_list

    json_write_path='./final_annot/data_with_keypoints/video_'+video_num+'.json'

    ##for writing json
#     with open(json_write_path,'w') as f:
#         json.dump(jaad_data,f,indent=2)


## Converting data to (300, 25, 2) and dataframe and saving csv as (15000,) df

In [255]:
## for not-crossing dataset
files_name=os.listdir('./final_annot/ncrossing_candidate/')
files_num=len(files_name)

ncrossing_df=pd.DataFrame(index=np.arange(0,158),columns=['keypoints','C/NC'])
ncrossing_df['C/NC']='NC'
index=0

ncrossing_json={}
for file_name in files_name:
    json_path='./final_annot/ncrossing_candidate/'+file_name
    with open(json_path,'r') as f:
        crossing_data=json.load(f)
    for person_id in crossing_data:
        keypoints_lists=[]
        if len(crossing_data[person_id])>300:
            frames_to_ignore=len(crossing_data[person_id])-300
        for frame in crossing_data[person_id]:
            if frames_to_ignore>0:
                frames_to_ignore-=1
                continue
            keypoints_lists.append(crossing_data[person_id][frame]['keypoints'])
        for _ in range(len(keypoints_lists),300):
            keypoints_lists.append(np.zeros((25,2)).tolist())
            
            
        keypoints_array=np.array(keypoints_lists)
        keypoints_array_flat=keypoints_array.flatten()
        ncrossing_json[index]=keypoints_array_flat.tolist()   
        
        ncrossing_df['keypoints'][index]=keypoints_array_flat
        index+=1

In [280]:
## for csv dump##
# np.set_printoptions(threshold=sys.maxsize)
# ncrossing_df.to_csv('./final_annot/csv_data/ncrossing.csv',index=False)

## for json dump##
# json_write_path='./final_annot/csv_data/ncrossing.json'

#     ##for writing json
# with open(json_write_path,'w') as f:
#     json.dump(ncrossing_json,f) #,indent=2)

In [265]:
len(ncrossing_json)

158

In [264]:
# for i in range(157):
#     #print(ncrossing_df['keypoints'][i].shape)
#     print(len(ncrossing_json[i]))

In [267]:
crossing_data

In [268]:
## for crossing dataset
files_name=os.listdir('./final_annot/crossing_candidate/')
files_num=len(files_name)

crossing_df=pd.DataFrame(index=np.arange(0,287),columns=['keypoints','C/NC'])
crossing_df['C/NC']='C'
index=0
frames_to_ignore=0
crossing_json={}
for file_name in files_name:
    json_path='./final_annot/crossing_candidate/'+file_name
    with open(json_path,'r') as f:
        crossing_data=json.load(f)
    for person_id in crossing_data:
        keypoints_lists=[]
        if len(crossing_data[person_id])>300:
            frames_to_ignore=len(crossing_data[person_id])-300
        for frame in crossing_data[person_id]:
            if frames_to_ignore>0:
                frames_to_ignore-=1
                continue
            keypoints_lists.append(crossing_data[person_id][frame]['keypoints'])
        for _ in range(len(keypoints_lists),300):
            keypoints_lists.append(np.zeros((25,2)).tolist())
            
        keypoints_array=np.array(keypoints_lists)
        keypoints_array_flat=keypoints_array.flatten()
        crossing_json[index]=keypoints_array_flat.tolist()
        
        crossing_df['keypoints'][index]=keypoints_array_flat
        index+=1

In [273]:
## for crossing json dump##
# crossing_df.to_csv('./final_annot/csv_data/crossing.csv',index=False)

## for ncrossing json dump##
json_write_path='./final_annot/csv_data/crossing.json'

    ##for writing json
with open(json_write_path,'w') as f:
    json.dump(crossing_json,f) #,indent=2)

In [269]:
len(crossing_json)

287

In [276]:
# for i in range(287):
#     print(crossing_df['keypoints'][i].shape)
#     #print(len(crossing_json[i]))

In [277]:
ncrossing_df

,keypoints,C/NC
0,"[568.542, 718.622, 592.123, 739.291, 603.817, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
153,"[0.0, 0.0, 951.211, 612.727, 954.079, 615.62, ...",NC
154,"[998.272, 680.349, 1001.15, 695.095, 986.477, ...",NC
155,"[0.0, 0.0, 1051.22, 727.479, 1062.96, 727.484,...",NC
156,"[998.272, 680.349, 1001.15, 695.095, 986.477, ...",NC


In [278]:
crossing_df

,keypoints,C/NC
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
4,"[1354.36, 648.205, 1357.44, 662.749, 1366.14, ...",C
...,...,...
282,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
283,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
284,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
285,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [166]:
c_nc_df

,keypoints,C/NC
0,"[0.0, 0.0, 500.893, 753.958, 506.685, 754.027,...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
440,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
442,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
443,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [298]:
c_nc_df=pd.concat([ncrossing_df,crossing_df],ignore_index=True)
# c_nc_df.to_csv('./final_annot/csv_data/c_nc.csv',index=False) #use pickle

In [155]:
# ncrossing_df.drop(158,axis=0,inplace=True)

## Converting zeros keypoints element to average 

In [300]:
# c_nc_df.to_pickle('./final_annot/pickle_json_data/c_nc_pkl.pkl')

In [516]:
temp_df_0=pd.read_pickle('./final_annot/pickle_json_data/c_nc_pkl.pkl')

In [517]:
temp_df_0

,keypoints,C/NC
0,"[568.542, 718.622, 592.123, 739.291, 603.817, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
440,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
442,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
443,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [445]:
temp_df

,keypoints,C/NC
0,"[568.542, 718.622, 592.123, 739.291, 603.817, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
440,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
442,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
443,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [395]:
test_array=temp_df['keypoints'][0].reshape(300,25,2)

In [316]:
# c_nc_df.to_pickle('./final_annot/pickle_json_data/temp.pkl')

In [312]:
np.set_printoptions(threshold=1000)

In [443]:
# test_array=temp_df['keypoints'][0].reshape(300,25,2)
for index in range(445):
    person_array=temp_df['keypoints'][index].reshape(300,25,2)
    for frame in range(300):
        array_per_frame=person_array[frame]
        if np.all(array_per_frame==0):
            continue
        if np.all(array_per_frame!=0):
            continue
        if np.all(array_per_frame[:,0] !=0) or np.all(array_per_frame[:,1] !=0):
            continue
        col_1_sum=np.sum(array_per_frame[:,0])
        col_2_sum=np.sum(array_per_frame[:,1])
        total_non_zero_element=np.sum(array_per_frame[:,0] != 0)
        col_1_avg=col_1_sum/total_non_zero_element
        col_2_avg=col_2_sum/total_non_zero_element
        person_array[frame][:,0]=(person_array[frame][:,0]==0)*col_1_avg+person_array[frame][:,0]
        person_array[frame][:,1]=(person_array[frame][:,1]==0)*col_2_avg+person_array[frame][:,1]
    person_array_flat=person_array.flatten()
    temp_df['keypoints'][index]=person_array_flat


In [444]:
temp_df.to_pickle('./final_annot/pickle_json_data/c_nc_averaged.pkl')

In [498]:
del_index

[164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 192,
 194,
 206,
 217,
 221,
 222,
 224,
 234,
 235,
 237,
 238,
 240,
 258,
 260,
 275,
 277,
 278,
 301,
 321,
 347,
 348,
 358,
 381,
 383,
 384,
 417,
 418]

In [523]:
temp_df=pd.read_pickle('./final_annot/pickle_json_data/c_nc_averaged.pkl')

In [524]:
temp_df

,keypoints,C/NC
0,"[568.542, 718.622, 592.123, 739.291, 603.817, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
440,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
442,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
443,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [525]:
temp_df.drop(del_index,axis=0,inplace=True)

In [526]:
temp_df.reset_index(inplace=True)

In [527]:
temp_df.drop(['index'],axis=1,inplace=True)

In [528]:
temp_df

,keypoints,C/NC
0,"[568.542, 718.622, 592.123, 739.291, 603.817, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
405,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
406,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
407,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
408,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [529]:
temp_df.to_pickle('./final_annot/pickle_json_data/c_nc_averaged_dropped_null.pkl')

In [442]:
person_array[0]

array([[568.542, 718.622],
       [592.123, 739.291],
       [603.817, 739.309],
       [615.56 , 765.802],
       [598.103, 786.37 ],
       [574.571, 739.363],
       [571.389, 771.65 ],
       [553.85 , 798.164],
       [594.982, 801.091],
       [600.955, 798.174],
       [601.004, 854.002],
       [600.983, 901.117],
       [577.467, 803.992],
       [574.393, 854.023],
       [571.533, 904.008],
       [571.533, 904.008],
       [571.395, 715.754],
       [597.95 , 715.76 ],
       [577.31 , 718.52 ],
       [556.834, 904.123],
       [553.904, 904.202],
       [574.428, 907.041],
       [595.071, 901.096],
       [603.848, 901.092],
       [601.018, 904.12 ]])

In [438]:
np.sum(person_array[290,:,1])/np.sum(person_array[290,:,1]!=0)

766.4016428571429

In [424]:
a=0
if np.all(test_array[a]==0):
    print('all-zero')
if np.all(test_array[a]!=0):
    print('all-not-zero')
if np.all(test_array[a,:,0] !=0) or np.all(test_array[a,:,1] !=0):
    print('one column not-zero')

all-not-zero
one column not-zero


In [328]:
test_array[0]=(test_array[0]==0)*-1+test_array[0]

In [430]:
test_array_0=test_array[0]
np.sum(test_array_0[:,0]!=0)

25

In [399]:
np.any(test_array[1]==0, axis=1)*2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [433]:
(test_array[50,:,1]==0)*3+test_array[50,:,1]

array([718.646, 739.237, 739.169, 765.707, 762.794, 739.353, 780.397,
       777.497, 806.854, 804.042, 853.979, 898.137, 806.936, 854.089,
       903.977,   3.   , 715.709, 715.726, 715.8  , 904.09 , 904.208,
       907.017, 898.226, 898.159, 903.961])

In [200]:
# c_nc_df['keypoints'][1].split('[')[1].split(']')[0].split()

['1448.57', '674.503', '1442.74', '...', '0.', '0.', '0.']

## Converting 300 dim to 200 removing zeros

In [3]:
df=pd.read_pickle('../pickle_json_data/c_nc_averaged_dropped_null.pkl')

In [65]:
df

,keypoints,C/NC
0,"[580.284, 721.597, 595.037, 742.143, 574.561, ...",NC
1,"[1448.57, 674.503, 1442.74, 698.071, 1422.12, ...",NC
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NC
3,"[998.371, 695.195, 1001.12, 712.743, 989.487, ...",NC
4,"[1110.14, 692.249, 1104.25, 701.048, 1095.41, ...",NC
...,...,...
405,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
406,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
407,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
408,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [85]:
df[df['C/NC']=='C']

,keypoints,C/NC
158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
159,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
160,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
161,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
162,"[1354.36, 648.205, 1357.44, 662.749, 1366.14, ...",C
...,...,...
405,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
406,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C
407,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C
408,"[409.747, 656.802, 406.572, 668.61, 406.676, 6...",C


In [63]:
np.array(list(df['keypoints'])).shape

(410, 10000)

In [70]:
index_fr_zeros=[]
index_fr_nonzero=[]
for i in range(410):
    if np.all(df['keypoints'][i].reshape(200,25,2)[100:200]==0):
        index_fr_zeros.append(i)
        continue
    index_fr_nonzero.append(i)

In [71]:
len(index_fr_zeros)

223

In [72]:
len(index_fr_nonzero)

187

In [60]:
temp=df['keypoints'][0].reshape(300,25,2)[100:300]

In [61]:
temp.flatten().shape

(10000,)

In [62]:
for i in range(410):
    if i in index_fr_zeros:
        new_array=df['keypoints'][i].reshape(300,25,2)[:200]
        new_array=new_array.flatten()
        df['keypoints'][i]=new_array
    if i in index_fr_nonzero:
        new_array=df['keypoints'][i].reshape(300,25,2)[100:300]
        new_array=new_array.flatten()
        df['keypoints'][i]=new_array


In [64]:
df.to_pickle('../pickle_json_data/c_nc_averaged_dropped_null_200.pkl')

## Testing

In [132]:
#for counting total person in json files
total=0
files_name=os.listdir('./final_annot/ncrossing_candidate/')
for file_name in files_name:
    json_path='./final_annot/ncrossing_candidate/'+file_name
    
    with open(json_path,'r') as f:
        data=json.load(f)
        a=len(data)
        #print(data.keys())
        total=total+a
    
print(total)

158


In [24]:
keypoints_arr=np.array(frame_list)

In [30]:
flatten_array=keypoints_arr.flatten()

In [31]:
new_array=flatten_array.reshape(300,25,2)

In [84]:
flatten_array.shape

(15000,)

In [58]:
###### for jaad json#####
video_num='0000'+str(2)
video_num=video_num[-4:]
json_path='./annot_json/video_'+video_num+'.json'

with open(json_path,'r') as f:
    jaad_data=json.load(f)

In [17]:
jaad_bboxes={}
for i in range (len(jaad_data)):
    frame_num=0
    keys=[*jaad_data]
    try:
        jaad_bbox=jaad_data[keys[i]][str(frame_num)]['bbox']
    except:
        continue
    jaad_bboxes[keys[i]]=jaad_bbox

In [14]:
#for keypoints file name
file_num='000000000000'+str(frame_num)
file_num=file_num[-12:]
file_name='./nbn_output/video_'+video_num+'/video_'+video_num+'_'+file_num+'_keypoints.json'

In [59]:
import os

listt = os.listdir('./nbn_output/video_'+video_num)
number_files = len(listt)
print (number_files)

210


In [250]:
for _ in range(10,9):
    print (1)

In [489]:
count_arr=[]
del_index=[]
for i in range(445):
    t_f=np.all(temp_df['keypoints'][i].reshape(300,25,2)[0:300]==0)
    if t_f==True:
        print(i)
        del_index.append(i)
    count_arr.append(t_f)


164
165
166
167
168
169
170
171
192
194
206
217
221
222
224
234
235
237
238
240
258
260
275
277
278
301
321
347
348
358
381
383
384
417
418


In [493]:
np.all(temp_df['keypoints'][416]==0)

False

In [73]:
temp_df=pd.read_pickle('../pickle_json_data/c_nc_dropped_null.pkl')

In [82]:
temp_df['keypoints'][temp_df['C/NC']=='NC']

0      [568.542, 718.622, 592.123, 739.291, 603.817, ...
1      [1448.57, 674.503, 1442.74, 698.071, 1422.12, ...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [998.371, 695.195, 1001.12, 712.743, 989.487, ...
4      [1110.14, 692.249, 1104.25, 701.048, 1095.41, ...
                             ...                        
153    [0.0, 0.0, 951.211, 612.727, 954.079, 615.62, ...
154    [998.272, 680.349, 1001.15, 695.095, 986.477, ...
155    [0.0, 0.0, 1051.22, 727.479, 1062.96, 727.484,...
156    [998.272, 680.349, 1001.15, 695.095, 986.477, ...
157    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: keypoints, Length: 158, dtype: object

## Adjacency matrix

In [ ]:
a_r=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [342]:
A=[    
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
    [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1],    
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0],    
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],    
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1]
    
]

In [347]:
modified_A=[    
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
    [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0.5],    
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0.5,  0,  0,  0],    
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],    
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0.5,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0.5,  1,  0,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0.5,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0.5,  0],    
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0.5,  1,  0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0.5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1]   
]

In [348]:
np.array(modified_A).shape

(25, 25)

In [387]:
i

4